# TODO

# Set-up

In [1]:
import os
import sys
import random
import time
import pandas as pd
import numpy as np

# ignore pandas warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/celloracle/bin/Bridge_Satpathy")
os.getcwd()

'/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/celloracle/bin/Bridge_Satpathy'

# Load file metadata

In [3]:
# Where the input metadata is stored for all files to run on
in_dir = "/cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127"

In [4]:
# Load the file metadata
file_metadata = pd.read_csv(os.path.join(in_dir, "file_metadata.tsv"), sep="\t")
file_metadata.head()

,file,file_type,cells,genes,normalization,n_cells,n_genes,percent_int,percent_0,percent_less_than_0
0,random_500.random_2000.raw.h5ad,.h5ad,random_500,random_2000,raw,500,2000,0.995209,0.951713,0.0
1,random_500.random_2000.raw.loom,.loom,random_500,random_2000,raw,500,2000,0.995209,0.951713,0.0
2,random_500.random_2000.raw.tsv,.tsv,random_500,random_2000,raw,2000,500,0.995209,0.951713,0.0
3,random_500.random_2000.log1p_cp10k.h5ad,.h5ad,random_500,random_2000,log1p_cp10k,500,2000,0.951713,0.951713,0.0
4,random_500.random_2000.log1p_cp10k.loom,.loom,random_500,random_2000,log1p_cp10k,500,2000,0.951713,0.951713,0.0


# Extract runs of interest

In [5]:
# Parameters for runs of interest
genes = ["0.05"]  # could be ["0.01", "0.05", "0.1", "0.2"]
cells = ["balanced_genotype_microglia"]
normalizations = ["cpm_rank"]  # could be [ "log1p_cp10k", "pf_log1p_pf", "cpm_rank"]
file_types = [".h5ad"]  # could be [".h5ad", ".loom", ".tsv"], but selecting tsv for GENIE3
out_dir = "/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/celloracle/results/Bridge_Satpathy"

In [6]:
# Set up masks for the file metadata based on above
genes_mask = file_metadata["genes"].isin(genes)
cells_mask = file_metadata["cells"].isin(cells)
normalizations_mask = file_metadata["normalization"].isin(normalizations)
file_types_mask = file_metadata["file_type"].isin(file_types)

In [7]:
# Extract the files to run
curr_metadata = file_metadata[genes_mask & cells_mask & normalizations_mask & file_types_mask]
curr_metadata["in_file"] = curr_metadata["cells"] + "/" + curr_metadata["genes"].astype(str) + "/" + curr_metadata["normalization"] + "/" + curr_metadata["file"]
curr_metadata["in_file_path"] = curr_metadata["in_file"].apply(lambda x: os.path.join(in_dir, x))
curr_metadata["genotype"] = [i.split(".")[0].split('-')[-1] for i in curr_metadata["file"]] 
curr_metadata["out_dir"] = out_dir
curr_metadata["out_dir_path"] = curr_metadata["out_dir"] + "/" + curr_metadata["cells"] + "/" + curr_metadata["genes"].astype(str) + "/" + curr_metadata["normalization"] + "/" + curr_metadata["genotype"]
curr_metadata

,file,file_type,cells,genes,normalization,n_cells,n_genes,percent_int,percent_0,percent_less_than_0,in_file,in_file_path,genotype,out_dir,out_dir_path
507,balanced_genotype_microglia-B6J.0.05.cpm_rank....,.h5ad,balanced_genotype_microglia,0.05,cpm_rank,1498,6277,0.010634,0.002413,0.498810,balanced_genotype_microglia/0.05/cpm_rank/bala...,/cellar/users/aklie/data/igvf/topic_grn_links/...,B6J,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/projects/igvf/topic_grn_li...
508,balanced_genotype_microglia-CASTJ.0.05.cpm_ran...,.h5ad,balanced_genotype_microglia,0.05,cpm_rank,1498,6277,0.010560,0.002531,0.498743,balanced_genotype_microglia/0.05/cpm_rank/bala...,/cellar/users/aklie/data/igvf/topic_grn_links/...,CASTJ,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/projects/igvf/topic_grn_li...
509,balanced_genotype_microglia-both.0.05.cpm_rank...,.h5ad,balanced_genotype_microglia,0.05,cpm_rank,2996,6277,0.010597,0.002472,0.498777,balanced_genotype_microglia/0.05/cpm_rank/bala...,/cellar/users/aklie/data/igvf/topic_grn_links/...,both,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/projects/igvf/topic_grn_li...


In [8]:
time_id = time.strftime("%Y%m%d-%H%M%S")
run_id =  random.getrandbits(32)
run = str(run_id) + "_" + time_id

In [9]:
curr_run = curr_metadata[["in_file_path", "out_dir_path"]]
curr_run["base_grn"] = "mouse_scATAC_atlas"
curr_run["bagging_number"] = 20
curr_run["alpha"] = 10
curr_run["layer"] = "normalized_counts"
curr_run["run_id"] = run
curr_run.to_csv(f"{run}_paths.csv", index=False)

# DONE

---